In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import scipy.stats as sts
import scipy
import seaborn as sns
import scipy.optimize as opt
from scipy.optimize import fmin
import seaborn as sns

In [2]:
df = pd.read_table('/content/IB_statistics_data_HW2.8.txt',sep='\s+', engine='python')

In [3]:
df

,Ex1,Ex1_group,Ex3,Ex3_group
0,-0.334665,1,15.166959,1
1,6.595572,1,15.342239,1
2,-1.285173,1,11.471799,1
3,9.011792,1,10.190140,1
4,2.784666,1,7.709481,1
...,...,...,...,...
203,8.299938,2,19.865967,4
204,7.265569,2,9.581659,4
205,1.810019,2,15.537624,4
206,5.870287,2,9.327303,4


### Задача 1.

In [4]:
X = df['Ex1'].to_numpy()
group = np.where(df['Ex1_group'].to_numpy() == 1)[0]
X1, X2 = X[group], X[group[-1]+1:]

Двухвыборочный t-тест (независимые выборки)

In [7]:
alpha = 0.05
n = len(X1) # так как выборки одного объема
T = (np.mean(X1) - np.mean(X2)) / (np.std(X1) + np.std(X2)) * np.sqrt(n) # статистика теста
fd = ((np.var(X1) + np.var(X2)) / n) ** 2 / ((np.var(X1) / n) **2 / (n - 1) + (np.var(X2) / n) **2 / (n - 1))
t = sts.t.cdf(x = T, df = fd)
tl, tu = sts.t.ppf(q = alpha / 2, df = fd), sts.t.ppf(q = 1 - alpha / 2, df = fd)
print(f'95%-доверительный интервал равен {tl}, {tu}')
print(f'Статистика критерия равна {t}')
if tl < t < tu:
    print(f'Нет оснований отклонить гипотезу с уровнем значимости {alpha}')
else:
    print(f'Есть основания отклонить гипотезу с уровнем значимости {alpha}')
print(f'p-value равно {2 * min(1 - sts.t.cdf(x = T, df = fd), sts.t.cdf(x = T, df = fd))}')

95%-доверительный интервал равен -1.9715496134232207, 1.9715496134232202
Статистика критерия равна 0.250418237591115
Нет оснований отклонить гипотезу с уровнем значимости 0.05
p-value равно 0.50083647518223


Двухвыборочный t-тест (зависимые выборки)

In [6]:
X12 = X1 - X2
T = np.sqrt(n) * np.mean(X12) / np.std(X12)
fd = n - 1
t = sts.t.cdf(x = T, df = fd)
tl, tu = sts.t.ppf(q = alpha / 2, df = fd), sts.t.ppf(q = 1 - alpha / 2, df = fd)
print(f'95%-доверительный интервал равен {tl}, {tu}')
print(f'Статистика критерия равна {t}')
if tl < t < tu:
    print(f'Нет оснований отклонить гипотезу с уровнем значимости {alpha}')
else:
    print(f'Есть основания отклонить гипотезу с уровнем значимости {alpha}')
print(f'p-value равно {2 * min(1 - sts.t.cdf(x = T, df = fd), sts.t.cdf(x = T, df = fd))}')

95%-доверительный интервал равен -1.9832641447097104, 1.98326414470971
Статистика критерия равна 0.1574506015676488
Нет оснований отклонить гипотезу с уровнем значимости 0.05
p-value равно 0.3149012031352976


### Задача 2.

*Теорема (Fisher–Cochran)*

Пусть $\xi$ ∼ $\mathcal{N}(0, \mathbb{I})$, а квадратичные формы $Q_1, . . . , Q_k$ рангов $n_1, . . . , n_k$ такие, что $ξ^Tξ = Q_1 + · · · + Q_k$. Тогда $n = \sum n_i$ равносильно $Q_i ∼ χ^2(n_i)$ и $Q_1, . . . , Q_k$ независимы.

*Доказательство*

Докажем сначала **достаточность**. Пусть выполнено $n = \sum n_i$. Поскольку  $ξ^Tξ = \sum Q_i$ и также $ξ^Tξ = \eta_1 + \eta_2 + ... \eta_n$, где $\sum \eta_i \sim χ^2(n)$ (так как $\xi_i$ ∼ $\mathcal{N}(0, 1)$ и независимы). Случай, когда $n_1 = ... = n_{k-1} = 0$ и $n_k = n$ был рассмотрен только что, когда есть всего одна квадратичная форма ($Q_k = \sum \eta_i$). Рассмотрим случай, когда есть несколько ненулевых рангов. В этом случае пусть у каждой квадратичной формы есть матрица $M_i$ и $\xi^T M_i \xi = Q_i$, тогда $\sum Q_i = \sum \xi^T M_i \xi = \xi^T (\sum  M_i) \xi = ξ^Tξ$ (по условию). То есть сумма $\sum  M_i= \mathbb{I}$. Значит мы можем диагонализировать все матрицы $M_i$. Пусть у нас есть частичная сумма матриц $M_i$ без $M_i$, то есть $S_i = \sum_\limits{j \neq i} M_j$. Тогда пусть мы диагонализировали матрицу $M_i$ и ее ранг равен $n_i$, значит она имеет $i$ ненулевых строк. Тогда ранг матрицы $S_i$ будет равен $n- n_i$. И эта матрица будет иметь $n-n_i$ ненулевых строк. В сумме ранг равен $n$. Теперь пусть мы выбрали такой базис, что ненулевые строки матриц $M_i$ и $S_i$ имеют различные номера, то есть ненулевые строки матрицы $S_i$ можно блоком вставить в то место, где стоят нули на диагонали матрицы $M_i$, то есть мы получили две независимые матрицы. Повторяя эту процедуру далее, получим $k$ независимых матриц, а, следовательно, и $k$ независимых квадратичных форм, отсюда и следует, что $Q_i \sim \chi^2(n_i)$ (потому что в диагонализированной матрице $M_i$ всего $n_i$ ненулевых строк.

Теперь докажем **необходимость**. Пусть у нас есть $k$ независимых квадратичных форм. Тогда зная о том, что мы можем диагонализировать их матрицы, сделаем действия, приведенные ранее, в обратном порядке. То есть если $Q_i \sim \chi^2(n_i)$, тогда матрица этой квадратичной формы имеет ранг $n_i$ (по определению ранга матрицы). Далее можем так же показать, что $\sum M_i = \mathbb{I}$. И, диагонализируя матрицы и выбирая нужный базис, получим, что можно составить блоки из ненулевых частей этих диагональных матриц, и далее получим $\sum n_i = n$.

### Задача 4.

- (a) Проверьте, что в случае двухфакторной модели дисперсионного анализа $R^2_1 = Q_0 + Q_α$.

Согласно лекции $Q_0 = \sum\limits_{i,j,k}(y_{ijk} − \hat μ − \hat α_i − \hat β_j −  \hat{(αβ)}_{ij})^2$ и $Q_α =\sum\limits_{i,j,k}(\hat α_i)^2$, тогда $Q_0 + Q_α = \sum\limits_{i,j,k}[(y_{ijk} − \hat μ − \hat α_i − \hat β_j −  \hat{(αβ)}_{ij})^2 + (\hat α_i)^2] = \sum\limits_{i,j,k}[(y_{ijk} − \hat μ − \hat α_i − \hat β_j −  \hat{(αβ)}_{ij}, y_{ijk} − \hat μ − \hat α_i − \hat β_j −  \hat{(αβ)}_{ij}) + (\hat α_i, \hat α_i)]$. Далее по свойству скалярного произведения получаем $\sum\limits_{i,j,k}[(y_{ijk} − \hat μ − \hat α_i − \hat β_j −  \hat{(αβ)}_{ij}, y_{ijk} − \hat μ − \hat α_i − \hat β_j −  \hat{(αβ)}_{ij}) + (\hat α_i, \hat α_i)] = \sum\limits_{i,j,k}[(y_{ijk} − \hat μ − \hat β_j −  \hat{(αβ)}_{ij}, y_{ijk} − \hat μ − \hat α_i − \hat β_j −  \hat{(αβ)}_{ij}) + (0, \hat α_i)] = \sum\limits_{i,j,k}[(y_{ijk} − \hat μ − \hat β_j −  \hat{(αβ)}_{ij}, y_{ijk} − \hat μ  − \hat β_j −  \hat{(αβ)}_{ij}) + (0, 0)] =  \sum\limits_{i,j,k}(y_{ijk} − \hat μ − \hat β_j −  \hat{(αβ)}_{ij}, y_{ijk} − \hat μ − \hat α_i − \hat β_j −  \hat{(αβ)}_{ij}) = \sum\limits_{i,j,k}(y_{ijk} − \hat μ − \hat β_j −  \hat{(αβ)}_{ij})^2 = R_1^2.$ Что и требовалось показать.

- (b) Придумайте (и докажите!) аналогичное разложение для проверки гипотезы $H_0 : (αβ)_{11} = · · · = (αβ)_{rs} = 0$.

Как и в предыдущем случае $Q_0 = \sum\limits_{i,j,k}(y_{ijk} − \hat μ − \hat α_i − \hat β_j −  \hat{(αβ)}_{ij})^2$, при этом $R_1^2 = \sum\limits_{i,j,k}(y_{ijk} − \hat μ − \hat α_i - \hat β_j)^2$, тогда $Q_{αβ} = \sum\limits_{i,j,k}(\hat{ αβ})_{ij}^2 = m\sum\limits_{j}^s\sum\limits_{i}^r(\bar y_{ij·} − \bar y_{i·} − \bar y_{·j} + \bar y)^2$. Тогда $\frac{Q_{αβ}}{σ} ∼ χ^2(r(s - 1))$ и $Q_0$ и $Q_{αβ}$ независимы, при этом в случае верной $H_0$ $\frac{Q_0}{σ} ∼ χ^2(rs(m-1))$, отсюда $F =\frac{rs(m−1)}{r(s−1)}·\frac{Q_{αβ}}{Q_0}∼ F(r(s − 1),rs(m − 1)).$ И искомое разложение имеет вид: $R^2_1 = Q_{αβ} + Q_0$. Далее сторим доверительный интервал и проверяем гипотезу.